<a href="https://colab.research.google.com/github/gzagarwal/GenerativeAi/blob/main/Next_Word_Prediction_Using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)  # force_remount to refresh login if needed

# 2. Import libraries
import os
from tensorflow.keras.models import load_model

# 3. Define the model path
model_path = '/content/drive/MyDrive/simple_rnn_imdb.h5'  # Change path if your file is in a different folder

# 4. Check if file exists
if os.path.exists(model_path):
    print(f"✅ Found model file at: {model_path}")
else:
    raise FileNotFoundError(f"❌ Model file not found at: {model_path}. Please upload it to your Drive.")

# 5. Load the model
try:
    model = load_model(model_path, compile=False)  # compile=False is safer if version mismatch
    print("✅ Model loaded successfully!")
    model.summary()
except Exception as e:
    print(f"❌ Failed to load model: {e}")


Mounted at /content/drive
✅ Found model file at: /content/drive/MyDrive/simple_rnn_imdb.h5
✅ Model loaded successfully!


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,025 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
## Data Collection

import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

#load the data

data=gutenberg.raw('shakespeare-hamlet.txt')

##save to a file

with open('hamlet.txt','w') as f:
  f.write(data)

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


In [5]:
## Data pre processing

import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from sklearn.model_selection import train_test_split

## load the dataset

with open('hamlet.txt','r') as file:
  text=file.read().lower()

## Tokenization the index created for words

tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1

total_words

4818

In [6]:
tokenizer.word_index

{'the': 1,
 'and': 2,
 'to': 3,
 'of': 4,
 'i': 5,
 'you': 6,
 'a': 7,
 'my': 8,
 'it': 9,
 'in': 10,
 'that': 11,
 'ham': 12,
 'is': 13,
 'not': 14,
 'his': 15,
 'this': 16,
 'with': 17,
 'your': 18,
 'but': 19,
 'for': 20,
 'me': 21,
 'lord': 22,
 'as': 23,
 'what': 24,
 'he': 25,
 'be': 26,
 'so': 27,
 'him': 28,
 'haue': 29,
 'king': 30,
 'will': 31,
 'no': 32,
 'our': 33,
 'we': 34,
 'on': 35,
 'are': 36,
 'if': 37,
 'all': 38,
 'then': 39,
 'shall': 40,
 'by': 41,
 'thou': 42,
 'come': 43,
 'or': 44,
 'hamlet': 45,
 'good': 46,
 'do': 47,
 'hor': 48,
 'her': 49,
 'let': 50,
 'now': 51,
 'thy': 52,
 'how': 53,
 'more': 54,
 'they': 55,
 'from': 56,
 'enter': 57,
 'at': 58,
 'was': 59,
 'oh': 60,
 'like': 61,
 'most': 62,
 'there': 63,
 'well': 64,
 'know': 65,
 'selfe': 66,
 'would': 67,
 'them': 68,
 'loue': 69,
 'may': 70,
 "'tis": 71,
 'vs': 72,
 'sir': 73,
 'qu': 74,
 'which': 75,
 'did': 76,
 'why': 77,
 'laer': 78,
 'giue': 79,
 'thee': 80,
 'ile': 81,
 'must': 82,
 'hath': 

In [7]:
## create input sequences

inputSequences=[]

for line in text.split('\n'):
  token_list=tokenizer.texts_to_sequences([line])[0]
  for i in range(1,len(token_list)):
      n_gram_sequence=token_list[:i+1]
      inputSequences.append(n_gram_sequence)


## pad sequences

max_sequence_len=max([len(x) for x in inputSequences]) ## it will max length
inputSequences=np.array(pad_sequences(inputSequences,maxlen=max_sequence_len,padding='pre'))
print(inputSequences)

[[   0    0    0 ...    0    1  687]
 [   0    0    0 ...    1  687    4]
 [   0    0    0 ...  687    4   45]
 ...
 [   0    0    0 ...    4   45 1047]
 [   0    0    0 ...   45 1047    4]
 [   0    0    0 ... 1047    4  193]]


In [8]:
## Create Predictors and Label

import tensorflow as tf
x,y=inputSequences[:,:-1],inputSequences[:,-1]


In [9]:
y=tf.keras.utils.to_categorical(y,num_classes=total_words)

In [10]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [20]:
## train our LSTM RNN

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout

model=Sequential()
model.add(Embedding(total_words,100))
model.build(input_shape=(None, max_sequence_len-1))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation='softmax'))

## compile the model

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

## train the model



model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ (None, 14, 100)        │       481,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 14, 150)        │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 14, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 100)            │       100,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4818)           │       486,618 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,219,418 (4.65 MB)

 Trainable params: 1,219,418 (4.65 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=1)

Epoch 1/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.0290 - loss: 7.1319 - val_accuracy: 0.0363 - val_loss: 6.6859
Epoch 2/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.0396 - loss: 6.4572 - val_accuracy: 0.0406 - val_loss: 6.7488
Epoch 3/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.0416 - loss: 6.3285 - val_accuracy: 0.0472 - val_loss: 6.7892
Epoch 4/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.0475 - loss: 6.2026 - val_accuracy: 0.0488 - val_loss: 6.8371
Epoch 5/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.0528 - loss: 6.0768 - val_accuracy: 0.0478 - val_loss: 6.8654
Epoch 6/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.0560 - loss: 5.9409 - val_accuracy: 0.0573 - val_loss: 6.8960
Epoch 7/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.0639 - loss: 5.8130 - val_accuracy: 0.0633 - val_loss: 6.9565
Epoch 8/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.0729 - loss: 5.6534 - va

In [23]:
#Function to predict the next word

def predict_next_word(model,tokenizer,text,max_sequence_len):
  toekn_list=tokenizer.texts_to_sequences([text])[0]
  toekn_list=pad_sequences([toekn_list],maxlen=max_sequence_len-1,padding='pre')
  prediction=model.predict(toekn_list)
  predicted_word_index=np.argmax(prediction,axis=1)
  for word,index in tokenizer.word_index.items():
    if index==predicted_word_index:
      return word
  return None

In [24]:
input_text="to be or not to be"
print(f"Input Text: {input_text}")
max_sequence_len=model.input_shape[1]+1
predicted_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Predicted Word: {predicted_word}")

Input Text: to be or not to be
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step
Predicted Word: remaines


In [26]:
model.save('/content/drive/MyDrive/next_word_lstm.h5')
import pickle
with open('/content/drive/MyDrive/tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [28]:
input_text="I haue seene"
print(f"Input Text: {input_text}")
max_sequence_len=model.input_shape[1]+1
predicted_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Predicted Word: {predicted_word}")

Input Text: I haue seene
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted Word: the


In [33]:
!pip install streamlit
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.3 MB/s eta 0:00:00


In [34]:
#Stream Lit App

import streamlit as st
import numpy as np
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [36]:
model=load_model('/content/drive/MyDrive/next_word_lstm.h5')
with open('/content/drive/MyDrive/tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)

#Function to predict the next word

def predict_next_word(model,tokenizer,text,max_sequence_len):
  toekn_list=tokenizer.texts_to_sequences([text])[0]
  toekn_list=pad_sequences([toekn_list],maxlen=max_sequence_len-1,padding='pre')
  prediction=model.predict(toekn_list)
  predicted_word_index=np.argmax(prediction,axis=1)
  for word,index in tokenizer.word_index.items():
    if index==predicted_word_index:
      return word
  return None


  #Streamlit app

  st.title("Next Word Prediction With LSTM ")

  input_text=st.text_input("Enter the text")
  st.button("Predict")

  if input_text:
    max_sequence_len=model.input_shape[1]+1
    predicted_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
    st.write(f"Predicted Word: {predicted_word}")

ERROR:pyngrok.process.ngrok:t=2025-04-27T19:03:08+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-04-27T19:03:08+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.